In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from sklearn.metrics import accuracy_score
import time

def load_dataset(file_path, text_column="text", label_column="label"):
    """โหลด dataset จากไฟล์"""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.json'):
        df = pd.read_json(file_path)
    elif file_path.endswith('.pkl'):
        df = pd.read_pickle(file_path)
    else:
        raise ValueError("Unsupported file format")

    # ตรวจสอบว่ามีคอลัมน์ที่จำเป็น
    if text_column not in df.columns or label_column not in df.columns:
        raise ValueError(f"Dataset must contain '{text_column}' and '{label_column}' columns")

    return df

def evaluate_single_model(model_name, model_path, dataset_paths, text_column="text", label_column="label"):
    """
    ประเมินโมเดลเดียวบนหลาย dataset
    Args:
        model_name: ชื่อโมเดลสำหรับแสดงผล
        model_path: path ของโมเดลจาก Hugging Face
        dataset_paths: dictionary ของ dataset {'dataset_name': 'file_path'}
        text_column: ชื่อคอลัมน์ข้อความใน dataset
        label_column: ชื่อคอลัมน์ label ใน dataset
    """
    print(f"\n{'='*50}")
    print(f"กำลังประเมินโมเดล: {model_name}")
    print(f"{'='*50}\n")

    # โหลดโมเดล
    start_time = time.time()
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, truncation=True)
        print(f"โหลดโมเดล {model_name} สำเร็จ (เวลาใช้: {time.time()-start_time:.2f} วินาที)")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดขณะโหลดโมเดล {model_name}: {str(e)}")
        return None

    # ประเมินบนแต่ละ dataset
    results = {}
    for ds_name, ds_path in dataset_paths.items():
        print(f"\nกำลังประเมินบน dataset: {ds_name}...")
        try:
            # โหลด dataset
            df = load_dataset(ds_path, text_column, label_column)
            total_rows = len(df)
            print(f"พบข้อมูลทั้งหมด {total_rows} แถว")

            # ทำนายและคำนวณ accuracy
            predictions = []
            true_labels = []

            for i, row in df.iterrows():
                try:
                    # แสดงความคืบหน้า ทุกๆ 10% ของข้อมูล
                    if (i+1) % max(1, total_rows//10) == 0 or (i+1) == total_rows:
                        print(f"กำลังประมวลผลแถวที่ {i+1}/{total_rows} [{(i+1)/total_rows*100:.1f}%]")

                    pred = pipeline(row[text_column])[0]["label"]
                    predictions.append(pred)
                    true_labels.append(row[label_column])

                    # แสดงผลลัพธ์ predicted พร้อมลำดับ
                    print(f"แถวที่ {i+1}: Predicted = {pred}, Actual = {row[label_column]}")

                except Exception as e:
                    print(f"Error processing row {i+1}: {e}")
                    continue

            accuracy = accuracy_score(true_labels, predictions)
            results[ds_name] = accuracy
            print(f"\nสรุปผลลัพธ์สำหรับ {ds_name}:")
            print(f"จำนวนข้อมูลทั้งหมด: {total_rows} แถว")
            print(f"จำนวนที่ทำนายสำเร็จ: {len(predictions)} แถว")
            print(f"Accuracy: {accuracy:.4f}")

        except Exception as e:
            print(f"เกิดข้อผิดพลาดขณะประเมินบน {ds_name}: {str(e)}")
            results[ds_name] = None

    return results

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    # กำหนดรายการ dataset (ชื่อ dataset: path ไฟล์)
    DATASETS = {
        'return_next': '/content/cleaned_dataset_0.csv',
        'dataset1': '/content/cleaned_dataset_1.csv',
        'dataset2': '/content/cleaned_dataset_2.csv',
        'dataset3': '/content/cleaned_dataset_3.csv',
        'dataset4': '/content/cleaned_dataset_4.csv',
        # 'Stock_Tweets': 'datasets/stock_tweets.xlsx',
        # 'Earnings_Calls': 'datasets/earnings_calls.json'
    }

    # กำหนดโมเดลที่ต้องการประเมิน (ชื่อโมเดล: path โมเดล)
    MODELS = {
        'Finbert_finetune_FinBERT_label': "Photchara/stock_sentiment_Finbert_label",
        'Bert_base_finetune_FinBERT_label': "Photchara/stock_sentiment_Finbert_v2",
        'Finbert_finetune_return_next': "Photchara/stock_sentiment_pct_change_v1",
        'Bert_base_finetune_return_next': "Photchara/stock_sentiment_pct_change_v2",
        # เพิ่มโมเดลอื่นๆ ที่นี่
    }

    # สร้าง DataFrame สำหรับเก็บผลลัพธ์
    results_df = pd.DataFrame(columns=DATASETS.keys())

    # ประเมินทีละโมเดล
    for model_name, model_path in MODELS.items():
        model_results = evaluate_single_model(model_name, model_path, DATASETS)

        if model_results:
            # เพิ่มผลลัพธ์ลงใน DataFrame
            results_df.loc[model_name] = model_results

    # แสดงผลลัพธ์ทั้งหมด
    print("\n\nสรุปผลการประเมินทั้งหมด:")
    print(results_df)

    # บันทึกผลลัพธ์
    results_df.to_csv('single_model_evaluation_results.csv')
    print("\nบันทึกผลลัพธ์ลงไฟล์ 'single_model_evaluation_results.csv' แล้ว")


กำลังประเมินโมเดล: Finbert_finetune_FinBERT_label



Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
แถวที่ 2240: Predicted = neutral, Actual = positive
แถวที่ 2241: Predicted = negative, Actual = positive
แถวที่ 2242: Predicted = positive, Actual = neutral
แถวที่ 2243: Predicted = negative, Actual = negative
แถวที่ 2244: Predicted = neutral, Actual = positive
แถวที่ 2245: Predicted = negative, Actual = neutral
แถวที่ 2246: Predicted = negative, Actual = positive
แถวที่ 2247: Predicted = neutral, Actual = positive
แถวที่ 2248: Predicted = neutral, Actual = positive
แถวที่ 2249: Predicted = neutral, Actual = positive
แถวที่ 2250: Predicted = neutral, Actual = positive
แถวที่ 2251: Predicted = neutral, Actual = positive
แถวที่ 2252: Predicted = negative, Actual = positive
แถวที่ 2253: Predicted = neutral, Actual = positive
แถวที่ 2254: Predicted = neutral, Actual = negative
แถวที่ 2255: Predicted = positive, Actual = positive
แถวที่ 2256: Predicted = neutral, Actual = positive
แถวที่ 2257: Predicted = neutral, Actual = positive
แถวที่ 2

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
แถวที่ 2251: Predicted = positive, Actual = positive
แถวที่ 2252: Predicted = positive, Actual = positive
แถวที่ 2253: Predicted = positive, Actual = positive
แถวที่ 2254: Predicted = positive, Actual = negative
แถวที่ 2255: Predicted = positive, Actual = positive
แถวที่ 2256: Predicted = positive, Actual = positive
แถวที่ 2257: Predicted = negative, Actual = positive
แถวที่ 2258: Predicted = positive, Actual = positive
แถวที่ 2259: Predicted = negative, Actual = positive
แถวที่ 2260: Predicted = positive, Actual = positive
แถวที่ 2261: Predicted = positive, Actual = neutral
แถวที่ 2262: Predicted = positive, Actual = neutral
แถวที่ 2263: Predicted = positive, Actual = neutral
แถวที่ 2264: Predicted = positive, Actual = neutral
แถวที่ 2265: Predicted = positive, Actual = positive
แถวที่ 2266: Predicted = positive, Actual = neutral
แถวที่ 2267: Predicted = positive, Actual = neutral
แถวที่ 2268: Predicted = positive, Actual = neutral
แถ

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from sklearn.metrics import accuracy_score
import time

def load_dataset(file_path, text_column="text", label_column="label"):
    """โหลด dataset จากไฟล์"""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.json'):
        df = pd.read_json(file_path)
    elif file_path.endswith('.pkl'):
        df = pd.read_pickle(file_path)
    else:
        raise ValueError("Unsupported file format")

    # ตรวจสอบว่ามีคอลัมน์ที่จำเป็น
    if text_column not in df.columns or label_column not in df.columns:
        raise ValueError(f"Dataset must contain '{text_column}' and '{label_column}' columns")

    return df

def evaluate_single_model(model_name, model_path, dataset_paths, text_column="text", label_column="label"):
    """
    ประเมินโมเดลเดียวบนหลาย dataset
    Args:
        model_name: ชื่อโมเดลสำหรับแสดงผล
        model_path: path ของโมเดลจาก Hugging Face
        dataset_paths: dictionary ของ dataset {'dataset_name': 'file_path'}
        text_column: ชื่อคอลัมน์ข้อความใน dataset
        label_column: ชื่อคอลัมน์ label ใน dataset
    """
    print(f"\n{'='*50}")
    print(f"กำลังประเมินโมเดล: {model_name}")
    print(f"{'='*50}\n")

    # โหลดโมเดล
    start_time = time.time()
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, truncation=True)
        print(f"โหลดโมเดล {model_name} สำเร็จ (เวลาใช้: {time.time()-start_time:.2f} วินาที)")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดขณะโหลดโมเดล {model_name}: {str(e)}")
        return None

    # ประเมินบนแต่ละ dataset
    results = {}
    for ds_name, ds_path in dataset_paths.items():
        print(f"\nกำลังประเมินบน dataset: {ds_name}...")
        try:
            # โหลด dataset
            df = load_dataset(ds_path, text_column, label_column)
            total_rows = len(df)
            print(f"พบข้อมูลทั้งหมด {total_rows} แถว")

            # ทำนายและคำนวณ accuracy
            predictions = []
            true_labels = []

            for i, row in df.iterrows():
                try:
                    # แสดงความคืบหน้า ทุกๆ 10% ของข้อมูล
                    if (i+1) % max(1, total_rows//10) == 0 or (i+1) == total_rows:
                        print(f"กำลังประมวลผลแถวที่ {i+1}/{total_rows} [{(i+1)/total_rows*100:.1f}%]")

                    pred = pipeline(row[text_column])[0]["label"]
                    predictions.append(pred)
                    true_labels.append(row[label_column])

                    # แสดงผลลัพธ์ predicted พร้อมลำดับ
                    print(f"แถวที่ {i+1}: Predicted = {pred}, Actual = {row[label_column]}")

                except Exception as e:
                    print(f"Error processing row {i+1}: {e}")
                    continue

            accuracy = accuracy_score(true_labels, predictions)
            results[ds_name] = accuracy
            print(f"\nสรุปผลลัพธ์สำหรับ {ds_name}:")
            print(f"จำนวนข้อมูลทั้งหมด: {total_rows} แถว")
            print(f"จำนวนที่ทำนายสำเร็จ: {len(predictions)} แถว")
            print(f"Accuracy: {accuracy:.4f}")

        except Exception as e:
            print(f"เกิดข้อผิดพลาดขณะประเมินบน {ds_name}: {str(e)}")
            results[ds_name] = None

    return results

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    # กำหนดรายการ dataset (ชื่อ dataset: path ไฟล์)
    DATASETS = {
        'return_next': '/content/cleaned_dataset_0.csv',
        'dataset1': '/content/cleaned_dataset_1.csv',
        'dataset2': '/content/cleaned_dataset_2.csv',
        'dataset3': '/content/cleaned_dataset_3.csv',
        'dataset4': '/content/cleaned_dataset_4.csv',
        # 'Stock_Tweets': 'datasets/stock_tweets.xlsx',
        # 'Earnings_Calls': 'datasets/earnings_calls.json'
    }

    # กำหนดโมเดลที่ต้องการประเมิน (ชื่อโมเดล: path โมเดล)
    MODELS = {
        'Finbert': "ProsusAI/finbert",
        # 'Finbert_finetune_FinBERT_label': "Photchara/stock_sentiment_Finbert_label",
        # 'Bert_base_finetune_FinBERT_label': "Photchara/stock_sentiment_Finbert_v2",
        # 'Finbert_finetune_return_next': "Photchara/stock_sentiment_pct_change_v1",
        # 'Bert_base_finetune_return_next': "Photchara/stock_sentiment_pct_change_v2",
        # เพิ่มโมเดลอื่นๆ ที่นี่
    }

    # สร้าง DataFrame สำหรับเก็บผลลัพธ์
    results_df = pd.DataFrame(columns=DATASETS.keys())

    # ประเมินทีละโมเดล
    for model_name, model_path in MODELS.items():
        model_results = evaluate_single_model(model_name, model_path, DATASETS)

        if model_results:
            # เพิ่มผลลัพธ์ลงใน DataFrame
            results_df.loc[model_name] = model_results

    # แสดงผลลัพธ์ทั้งหมด
    print("\n\nสรุปผลการประเมินทั้งหมด:")
    print(results_df)

    # บันทึกผลลัพธ์
    results_df.to_csv('single_model_evaluation_results.csv')
    print("\nบันทึกผลลัพธ์ลงไฟล์ 'single_model_evaluation_results.csv' แล้ว")


กำลังประเมินโมเดล: Finbert



tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

โหลดโมเดล Finbert สำเร็จ (เวลาใช้: 6.90 วินาที)

กำลังประเมินบน dataset: return_next...
พบข้อมูลทั้งหมด 5123 แถว
แถวที่ 1: Predicted = neutral, Actual = positive
แถวที่ 2: Predicted = neutral, Actual = positive
แถวที่ 3: Predicted = neutral, Actual = positive
แถวที่ 4: Predicted = neutral, Actual = positive
แถวที่ 5: Predicted = negative, Actual = positive
แถวที่ 6: Predicted = negative, Actual = positive
แถวที่ 7: Predicted = negative, Actual = positive
แถวที่ 8: Predicted = positive, Actual = positive
แถวที่ 9: Predicted = positive, Actual = positive
แถวที่ 10: Predicted = positive, Actual = neutral
แถวที่ 11: Predicted = negative, Actual = neutral
แถวที่ 12: Predicted = negative, Actual = neutral
แถวที่ 13: Predicted = negative, Actual = neutral
แถวที่ 14: Predicted = negative, Actual = neutral
แถวที่ 15: Predicted = neutral, Actual = neutral
แถวที่ 16: Predicted = neutral, Actual = neutral
แถวที่ 17: Predicted = negative, Actual = neutral
แถวที่ 18: Predicted = negative, Actual = n

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


แถวที่ 25: Predicted = positive, Actual = neutral
แถวที่ 26: Predicted = neutral, Actual = neutral
แถวที่ 27: Predicted = negative, Actual = neutral
แถวที่ 28: Predicted = neutral, Actual = neutral
แถวที่ 29: Predicted = neutral, Actual = neutral
แถวที่ 30: Predicted = negative, Actual = neutral
แถวที่ 31: Predicted = negative, Actual = neutral
แถวที่ 32: Predicted = neutral, Actual = neutral
แถวที่ 33: Predicted = negative, Actual = neutral
แถวที่ 34: Predicted = neutral, Actual = neutral
แถวที่ 35: Predicted = negative, Actual = neutral
แถวที่ 36: Predicted = neutral, Actual = neutral
แถวที่ 37: Predicted = negative, Actual = neutral


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
แถวที่ 2242: Predicted = positive, Actual = neutral
แถวที่ 2243: Predicted = negative, Actual = negative
แถวที่ 2244: Predicted = neutral, Actual = positive
แถวที่ 2245: Predicted = negative, Actual = neutral
แถวที่ 2246: Predicted = negative, Actual = positive
แถวที่ 2247: Predicted = neutral, Actual = positive
แถวที่ 2248: Predicted = neutral, Actual = positive
แถวที่ 2249: Predicted = neutral, Actual = positive
แถวที่ 2250: Predicted = neutral, Actual = positive
แถวที่ 2251: Predicted = neutral, Actual = positive
แถวที่ 2252: Predicted = negative, Actual = positive
แถวที่ 2253: Predicted = positive, Actual = positive
แถวที่ 2254: Predicted = neutral, Actual = negative
แถวที่ 2255: Predicted = positive, Actual = positive
แถวที่ 2256: Predicted = neutral, Actual = positive
แถวที่ 2257: Predicted = neutral, Actual = positive
แถวที่ 2258: Predicted = positive, Actual = positive
แถวที่ 2259: Predicted = neutral, Actual = positive
แถวที่ 